In [12]:
import torch
import pyro
import numpy as np
from torch.nn import ReLU, LeakyReLU
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm import tqdm

# Simple model with one hidden layer
from src.models.regression import MLERegression
from src.metrics.regression import  RootMeanSquaredError, PCIP, MPIW
# Six UCI datasets: concrete, power, energy, wine, yacht, housing
from src.data.regression import UCI
from src.commons.pyro_training import to_bayesian_model
from src.commons.utils import seed_everything

In [13]:
input_sizes = {
    'concrete': 7,
    'power': 3,
    'energy': 8,
    'wine': 10,
    'yacht': 5,
    'housing': 12
}

relu = ReLU()
leaky = LeakyReLU(negative_slope = 0.5)

In [14]:
# Reference example: http://pyro.ai/examples/bayesian_regression.html

def training(svi, train_loader, epoch, device):
    loss = 0
    for idx, (X, y) in tqdm(
        enumerate(train_loader), total=len(train_loader), desc=f"Training epoch {epoch}", miniters=10
    ):
        X = X.to(device)
        y = y.to(device)
        step_loss = svi.step(X, y)
        loss += step_loss
        batch_index = (epoch + 1) * len(train_loader) + idx
    return loss

def evaluation(predictive, dataloader, metrics, device):
    for idx, (X, y) in tqdm(enumerate(dataloader), desc=f"Evaluation", miniters=10):
        if not np.isfinite(np.array(X)).all():
            print("WARNING: INF IN DATA")
            X[X == float("-INF")] = 0
            X[X == float("INF")] = 0
            
        y = y.to(device)
        out = predictive(X.to(device))["obs"].T
        for metric in metrics.values():
            metric.update(out, y)


In [15]:
def train_loop(
    model,
    guide,
    train_loader,
    valid_loader,
    svi,
    epochs,
    num_samples,
    metrics,
    device,
):
    
    for e in range(epochs):
        loss = training(svi, train_loader, e, device)
        print(f"Loss: {loss}")
        
        predictive = Predictive(model, guide=guide, num_samples=num_samples, return_sites=("obs",))
        evaluation(predictive, valid_loader, metrics, device)
        
        for metric in metrics:
            print(f"{metric}: {metrics[metric].compute().cpu()}")
            metrics[metric].reset()
            
    return model, guide

In [16]:
def bayesian_training(
    seed,
    max_epochs,
    num_samples,
    batch_size,
    task,
    device,
    optimizer,
    criterion,
    sigma_bound,
    param_mean,
    param_std,
    hidden_size,
    activation
):
    seed_everything(seed)
    pyro.clear_param_store()

    net = MLERegression(activation=activation, in_size=input_sizes[task], hidden_size=hidden_size, out_size=1)
# the model is converted using code in src.models.bnn, in particular BNNRegression class defining bayesian model
    model = to_bayesian_model(net, param_mean, param_std, device=device, sigma_bound=sigma_bound)
    svi = SVI(model.model, model.guide, optimizer, loss=criterion)
    
    dataloader_args = {
      "num_workers": 4,
      "pin_memory": True,
      "persistent_workers": True
    }
    
    uci = UCI(task, train_batch_size=batch_size, test_batch_size=batch_size, train_ratio=0.8, validation_ratio=0.2, dataloader_args=dataloader_args)
    train_loader = uci.train_dataloader()
    valid_loader = uci.validation_dataloader()
    
    metrics = {
    "RMSE": RootMeanSquaredError(input_type="samples"),
    "PCIP": PCIP(input_type="none", percentile=80),
    "MPIW": MPIW(input_type="none", percentile=80)
    }

    for metric in metrics.values():
        metric.set_device(device)
        
    return train_loop(model.model, model.guide, train_loader, valid_loader, svi, max_epochs, num_samples, metrics, device)

In [17]:
param_dict = {
    "seed": 42,
    "max_epochs": 50,
    "num_samples": 50,
    "batch_size": 32,
    "task": 'housing',
    "device": torch.device("cuda"),

    "optimizer": pyro.optim.Adam({"lr": 0.001 ,"betas": [0.9, 0.999]}),
    "criterion": Trace_ELBO(),

    # The models output is Normal(mean, std) where std has prior Uniform(0, sigma_bound)
    "sigma_bound": 5.0,

    # mean and std of bayesian model priors
    "param_mean": 0.0,
    "param_std": 1.0,

    "hidden_size": 40,
    "activation": relu
}

The data is normalized so RMSE of naive model is ~1. 

PCIP metric is adjusted for percentile so for example for 80% confidence interval value 5 means that 85% of datapoint fall into the interval and value -3 means that 77% of the datapoints fall into the interval. In other words, the closer the PCIP metric is to 0 the better.

Here we can see that the model starts with relatively reasonable PCIP but as the model trains it goes up to 18 indicating that 98% of datapoints fall into 80% confidence interval (despite the fact that RMSE goes down so the model converges). 

By changing param_std, sigma_bound, learning_rate, batch_size or the shape of the net one can affect the course of training so that the initial PCIP can be negative/positive and increase/decrease throughout training. But I didn't manage to set the parameters in such a way that it converges to 0.

In [18]:
bayesian_training(**param_dict)

Train data shape
torch.Size([404, 12])
torch.Size([404])
Test data shape
torch.Size([101, 12])
torch.Size([101])
NAIVE RMSE: 0.9948453971905475


Training epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 62.86it/s]

Loss: 12201.28772854805



Evaluation: 3it [00:00,  5.01it/s]


RMSE: 1.9745211601257324
PCIP: 3.75
MPIW: 5.2330827713012695


...

Training epoch 49: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 67.62it/s]


Loss: 8822.7569039464


Evaluation: 3it [00:00,  5.61it/s]


RMSE: 0.6290941834449768
PCIP: 18.425003051757812
MPIW: 4.875285625457764


(BNNRegression(
   (model): PyroMLERegression(
     (activation): PyroReLU()
     (layer1): PyroLinear(in_features=12, out_features=40, bias=True)
     (layer3): PyroLinear(in_features=40, out_features=1, bias=True)
   )
 ),
 AutoDiagonalNormal())